# Notebook on Recommender Systems

In this Notebook I will explore the subject of recommender systems. The goal will be to understand how a recommender system works, and after that, to build a simple one.

## 1 Building a simple Recommender system

To start I will build a simple collaborative filtering algorithm that will recommend new items to a user by finding similar items or similar users.

### 1.1 Dataset

To do testing, I decided to use a movie dataset, containing a list of movies, users and ratings.

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History
and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4,
Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872

Let's explore this dataset !

In [2]:
import pandas as pd

ratings_data = pd.read_csv("datasets/movies/ratings.dat", delimiter="::", engine='python', encoding='latin1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

ratings_data.head()


UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

This file contains the data about the ratings, the first column is the userid, the second one is the id of the movie, the third one is the rating ($r$) such that $$r \in \{ n \in \mathbb{Z}| 1 \leq r \leq 5 \} $$

We also have access to the timestamp from when the rating was published by the user.

In [4]:
movies_data = pd.read_csv("datasets/movies/movies.dat", delimiter="::", engine='python', encoding='latin1', header=None, names=['MovieID', 'Title', 'Genres'])
movies_data.head()

MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy

For the movies data, like the ratings data, the format is the same, column are separated using the "::" separator, and the genres for the movies are separated by "|". There is a thirs file, we will, come back to it later.

### 1.2 User-based collaborative filtering

Now that we explored our dataset, the first implementation will be User-based, meaning that we will try to recommend a movie to a user by comparing users ratings on certain movies.

To do that, I will use cosine similarity, and then retrieve the closest neighbors to get similar items.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# first let's create a matrix, the rows will represent each user, and the each column a movie,
# and then the value represents the rating the user gave to that movie (0 = no rating given)

ratings_matrix = ratings_data.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=0)

ratings_matrix.head()

MovieID  1     2     3     4     5     6     7     8     9     10    ...  \
UserID                                                               ...   
1         5.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
5         0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0  ...   

MovieID  3943  3944  3945  3946  3947  3948  3949  3950  3951  3952  
UserID                                                               
1         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
5         0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 3706 columns]

What we did here is pivot the dataframe to create a 2D array (matrix), where each index/row is the id of each users (they range from 1 to 6040) and each column is a movie id (ranging from 1 to 3952). Each values placed are the rating for each movie and user, the default value in case a user didn't post any review for a film is 0.

In [18]:
# Now we compute the consine similarity between users

user_similarity = cosine_similarity(ratings_matrix)

user_similarity

array([[1.        , 0.09638153, 0.12060981, ..., 0.        , 0.17460369,
        0.13359025],
       [0.09638153, 1.        , 0.1514786 , ..., 0.06611767, 0.0664575 ,
        0.21827563],
       [0.12060981, 0.1514786 , 1.        , ..., 0.12023352, 0.09467506,
        0.13314404],
       ...,
       [0.        , 0.06611767, 0.12023352, ..., 1.        , 0.16171426,
        0.09930008],
       [0.17460369, 0.0664575 , 0.09467506, ..., 0.16171426, 1.        ,
        0.22833237],
       [0.13359025, 0.21827563, 0.13314404, ..., 0.09930008, 0.22833237,
        1.        ]])

For each user, we have a score for every other user that represents how similar they are. The closer the cosine value is to 1, the more similar they are.

In [20]:
# for example let's see how similar the user with the id 1 is to itself
user_similarity[1][1]

1.0

Indeed, if we compare the same user, we can see that they are completly identical. Now that we established this matrix, we can compute for each user his closest neighbor(s)

In [38]:
import numpy as np

num_users = user_similarity.shape[0]  # Number of users
N = 5  # Number of nearest neighbors to find, excluding the user itself

nearest_neighbors = []

for user_idx in range(num_users):
    # Get the similarity scores for the current user and sort them in descending order
    sorted_user_similarities = np.argsort(-user_similarity[user_idx])
    
    # Select the top N+1 scores (including the user itself) and exclude the user's own index
    top_user_neighbors = sorted_user_similarities[1:N+1]
    
    # Append the indices (user IDs) of the top N neighbors to the list
    nearest_neighbors.append(top_user_neighbors)

# Convert the list to a numpy array for easier handling
nearest_neighbors = np.array(nearest_neighbors)

# Now, `nearest_neighbors` contains the indices of the N nearest neighbors for each user

nearest_neighbors


array([[5342, 5189, 1480, 1282, 5704],
       [3107,   94, 2813, 4600, 2302],
       [2999,  478, 5690, 3499, 1903],
       ...,
       [2208, 3582, 4653,  757, 2798],
       [ 930, 3024, 4853, 1321, 4929],
       [1631, 1898, 2543, 3370,  587]], dtype=int64)

With the code above, we now know for each user what are their top five similar users.